In [1]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

Now check all our dependencies. If Triton is not compatible with the GPU or the CUDA runtime served by Colab, please make sure that it's not installed in the above

In [3]:
import math
import os

import pytorch_lightning as pl
import torch
import torch.nn as nn
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.utilities import rank_zero_info
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset, RandomSampler

from xformers.factory.model_factory import xFormer, xFormerConfig

/root/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/root/miniconda3/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Either FairScale or torch distributed is not available, MixtureOfExperts will not be exposed. Please install them if you would like to use MoE


Let's first define our GPT-like model. Please note that all the parameters in the config dictionnary can be changed more or less at will, but the attention mechanism needs to be compatible with causality constraints. We'll be using Pytorch Lightning to nicely specify all the specific training steps

In [4]:
class GPT(pl.LightningModule):
    """  the full GPT language model, with a context size of block_size """

    def __init__(
        self,
        vocab_size,
        weight_decay=0.1,
        betas=(0.9, 0.95),
        learning_rate=1e-4,
        n_embd=512,
        block_size=128,
        n_layer=8,
        n_head=4,
        resid_pdrop=0.1,
        attn_pdrop=0.1,
        mlp_pdrop=0.1,
        attention="fourier_mix",
        hidden_layer_multiplier=4,
        warmup_tokens=20,
        final_tokens=1000,
    ):
        super().__init__()

        # auto creates self.hparams from the method signature
        self.save_hyperparameters()

        # A list of the encoder or decoder blocks which constitute the Transformer.
        xformer_config = [
            {
                "block_type": "encoder",
                "num_layers": self.hparams.n_layer,
                "dim_model": self.hparams.n_embd,
                "residual_norm_style": "pre",
                "position_encoding_config": {
                    "name": "vocab",
                    "seq_len": self.hparams.block_size,
                    "vocab_size": self.hparams.vocab_size,
                },
                "multi_head_config": {
                    "num_heads": self.hparams.n_head,
                    "residual_dropout": self.hparams.resid_pdrop,
                    "use_rotary_embeddings": True,
                    "attention": {
                        "name": self.hparams.attention,
                        "dropout": self.hparams.attn_pdrop,
                        "causal": True,
                        "seq_len": self.hparams.block_size,
                    },
                },
                "feedforward_config": {
                    "name": "MLP",
                    "dropout": self.hparams.mlp_pdrop,
                    "activation": "gelu",
                    "hidden_layer_multiplier": self.hparams.hidden_layer_multiplier,
                },
            }
        ]

        config = xFormerConfig(xformer_config)
        self.model = xFormer.from_config(config)

        # decoder head
        self.ln_f = nn.LayerNorm(self.hparams.n_embd)
        self.head = nn.Linear(self.hparams.n_embd, self.hparams.vocab_size, bias=False)

        self.block_size = self.hparams.block_size
        self.apply(self._init_weights)

        self._tokens_seen = 0

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

        # Reset the token counter
        self._tokens_seen = 0

    def get_block_size(self):
        return self.block_size

    def configure_optimizers(self):
        # Create the optimizer and the training schedule:
        # - Handle the per-param weight decay
        no_decay = ["bias", "LayerNorm.weight"]
        params_decay = [
            p for n, p in self.named_parameters() if not any(nd in n for nd in no_decay)
        ]
        params_nodecay = [
            p for n, p in self.named_parameters() if any(nd in n for nd in no_decay)
        ]
        optim_groups = [
            {"params": params_decay, "weight_decay": self.hparams.weight_decay},
            {"params": params_nodecay, "weight_decay": 0.0},
        ]

        # - Start with a warm up, ramp up then cosine
        optimizer = torch.optim.AdamW(
            optim_groups, lr=self.hparams.learning_rate, betas=self.hparams.betas
        )

        def update_lr(*_):
            config = self.hparams

            if self._tokens_seen < config.warmup_tokens:
                # linear warmup
                lr_mult = float(self._tokens_seen) / float(max(1, config.warmup_tokens))
                lr_mult = max(lr_mult, 1e-2)  # could be that we've not seen any yet
            else:
                # cosine learning rate decay
                progress = float(self._tokens_seen - config.warmup_tokens) / float(
                    max(1, config.final_tokens - config.warmup_tokens)
                )
                lr_mult = max(0.1, 0.5 * (1.0 + math.cos(math.pi * progress)))

            return lr_mult

        lr_scheduler = {
            "scheduler": torch.optim.lr_scheduler.LambdaLR(
                optimizer,
                lr_lambda=[update_lr, update_lr],
            ),
            "name": "learning_rate",
            "interval": "step",  # The unit of the scheduler's step size
            "frequency": 1,  # The frequency of the scheduler
        }
        return [optimizer], [lr_scheduler]

    def forward(self, src):
        # predict the next tokens (in latent space)
        prediction = self.model(src)

        # translate the predictions into tokens
        prediction = self.ln_f(prediction)
        logits = self.head(prediction)

        return logits

    def training_step(self, batch, _):
        src, targets = batch

        # Update the tokens we've seen (tracked for LR scheduling)
        self._tokens_seen += (src >= 0).numel()

        # same action as inference
        logits = self(src)

        # if we are given some desired targets also calculate the loss
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))

        self.logger.log_metrics(
            {
                "train_loss": loss.mean(),
                "learning_rate": self.lr_schedulers().get_last_lr()[0],
            },
            step=trainer.global_step,
        )

        return loss

Now let's define our dataset. This comes straight from MinGPT, and the idea is to serve a sequence of character (of size `block_size`) given any starting point

In [5]:
class CharDataset(Dataset):
    def __init__(self, data, block_size):
        chars = list(set(data))
        data_size, vocab_size = len(data), len(chars)
        rank_zero_info("data has %d characters, %d unique." % (data_size, vocab_size))

        self.stoi = {ch: i for i, ch in enumerate(chars)}
        self.itos = {i: ch for i, ch in enumerate(chars)}
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data

    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, i):
        chunk = self.data[i : i + self.block_size + 1]
        dix = [self.stoi[s] for s in chunk]

        # src and target are off by one, we want the model to predict the next word
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y

    def to_tokens(self, message, device):
        return torch.tensor([self.stoi[s] for s in message], dtype=torch.long)[
            None, ...
        ].to(device)

    def from_tokens(self, tokens):
        return "".join([self.itos[int(i)] for i in tokens])

Now, let's plan ahead: how can we probe our model ? Given the training (guess the next character), a nice way is to sample the model given an initial bait. The predictions are then chained after the bait, and we can keep probing the model for predictions over a rolling window. Note that contrary to the training phase, this is sequential, we only predict one character ahead and then repeat

In [6]:
@torch.no_grad()
def sample(model, x, steps, temperature=1.0, sample=False, top_k=None):
    """
    take a conditioning sequence of indices in x (of shape (b,t)) and predict the next token in
    the sequence, feeding the predictions back into the model each time. Clearly the sampling
    has quadratic complexity unlike an RNN that is only linear, and has a finite context window
    of block_size, unlike an RNN that has an infinite context window.
    """
    block_size = model.get_block_size()
    model.eval()

    # CREDITS: https://github.com/karpathy/minGPT/blob/master/mingpt/utils.py
    def top_k_logits(logits, k):
        v, _ = torch.topk(logits, k)
        out = logits.clone()
        out[out < v[:, [-1]]] = -float("Inf")
        return out

    for _ in range(steps):
        x_cond = (
            x if x.size(1) <= block_size else x[:, -block_size:]
        )  # crop context if needed
        logits = model(x_cond)

        # pluck the logits at the final step and scale by temperature
        logits = logits[:, -1, :] / temperature

        # optionally crop probabilities to only the top k options
        if top_k is not None:
            logits = top_k_logits(logits, top_k)

        # apply softmax to convert to probabilities
        probs = F.softmax(logits, dim=-1)

        # sample from the distribution or take the most likely
        if sample:
            ix = torch.multinomial(probs, num_samples=1)
        else:
            _, ix = torch.topk(probs, k=1, dim=-1)

        # append to the sequence and continue
        x = torch.cat((x, ix), dim=1)

    return x[0]  # escape the batch dimension

Ok, good to go, we're equipped ! Let's train a model. Feel free to alter the parameters to get a feel of what's right or wrong


In [8]:
seed_everything(42)
REF_BATCH = 512
BATCH = 32  # adjust depending on the available memory on your machine
WORKERS = 2
EPOCHS = 1
BLOCK = 128
WARMUP = 20
LR = 6e-4
LAYERS = 4

if not os.path.exists("input.txt"):
    os.system(
        "wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
    )

text = open("input.txt", "r").read()
train_dataset = CharDataset(
    text, BLOCK
)  # one line of poem is roughly 50 characters
random_sampler = RandomSampler(train_dataset)
train_loader = DataLoader(
    train_dataset,
    sampler=random_sampler,
    batch_size=BATCH,
    num_workers=WORKERS,
    pin_memory=True,
)

model = GPT(
    vocab_size=train_dataset.vocab_size,
    block_size=train_dataset.block_size,
    attention="scaled_dot_product",
    warmup_tokens=REF_BATCH * WARMUP,
    learning_rate=LR,
    final_tokens=EPOCHS * len(train_dataset) * BLOCK,
    n_layer=LAYERS
)

trainer = Trainer(
    devices=1, accelerator="gpu",
    max_epochs=EPOCHS,
    precision=32,
    gradient_clip_val=1,
    log_every_n_steps=1,
    detect_anomaly=True,
    accumulate_grad_batches=REF_BATCH // BATCH,
)

trainer.fit(model, train_loader)

Seed set to 42
data has 1115394 characters, 65 unique.
/root/miniconda3/lib/python3.10/site-packages/xformers/factory/model_factory.py:105: FutureWarning: xFormerConfig(stack_configs=[xFormerEncoderConfig(dim_model=512, feedforward_config=MlpConfig(name='MLP', dim_model=512, dropout=0.1, activation='gelu', hidden_layer_multiplier=4, bias=None), position_encoding_config=VocabEmbeddingConfig(name='vocab', dim_model=512, seq_len=128, vocab_size=65, dropout=None), block_type=<BlockType.Encoder: 'encoder'>, residual_norm_style=<ResidualNormStyle.Pre: 'pre'>, normalization=<NormalizationType.LayerNorm: 'layernorm'>, layer_position=<xformers.factory.block_configs.LayerPosition object at 0x7fe72c56eb30>, use_triton=True, reversible=False, num_layers=4, multi_head_config={'num_heads': 4, 'residual_dropout': 0.1, 'use_rotary_embeddings': True, 'attention': {'name': 'scaled_dot_product', 'dropout': 0.1, 'causal': True, 'seq_len': 128}, 'dim_model': 512}, simplicial_embeddings=None, patch_embeddin

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [15]:
context = "Out, out, brief candle! Life's but a walking shadow"  # 数据集来自《悲剧》--莎士比亚
x = train_dataset.to_tokens(context, model.device)
y = sample(model, x, steps=1000, temperature=1.0, sample=True, top_k=10)

print(train_dataset.from_tokens(y))

Out, out, brief candle! Life's but a walking shadow.

LUCENTIO:
How now, mistrust? Good my most sovereign,
And make his meat that worthy danger but
A craft of this most guard to us.

CORIOLANUS:
Now, most well-spare not.

CORIOLANUS:
Mended upon a prince,
More than my head is soldiers.

SICINIUS:
He's coming.

CORIOLANUS:
Well, sweet masters!
The prince's doom my father shall we on.

Second Servingman:
And I do discharge thee?

CAMILLO:
He hath brought my head; and let him desire
His barbarism with his beggar's shop impossible
To unsubstantial drums; then she is accept
The white o' the fire, it is all on the field
That truth to company specially. Therefore, my
sweet lords, to hunger about my grave!
An eye o' the way is setten heir,
As it are clear'd for the fashion's house,
Now take her brothers, with this feast, having worm,
That will our coastle could be advised?

LADY CAPULET:
Why, how began; whom we hear like thee in some pity
that thou speak'st thy noble teeth of my soul.

JULIET: